# K-Means 聚类算法

**核心概念**: K-Means 是最经典的划分式聚类算法，通过迭代优化将数据划分为 K 个簇

## 算法原理

K-Means 的目标是最小化簇内平方误差 (Within-Cluster Sum of Squares, WCSS):

$$J = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2$$

其中 $\mu_i$ 是簇 $C_i$ 的质心。

### 算法步骤

1. **初始化**: 随机选择 K 个点作为初始质心 (或使用 k-means++ 初始化)
2. **分配**: 将每个样本分配给距离最近的质心所在的簇
3. **更新**: 重新计算每个簇的质心 (簇内所有点的均值)
4. **迭代**: 重复步骤 2-3 直到质心不再变化或达到最大迭代次数

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris, make_blobs
from sklearn.preprocessing import StandardScaler

# 设置随机种子以确保结果可复现
np.random.seed(42)

# 配置 matplotlib 支持中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 数据准备

使用鸢尾花数据集进行演示，该数据集包含 150 个样本，4 个特征，3 个类别。

In [ ]:
# 加载鸢尾花数据集
iris = load_iris()
X = iris.data
y_true = iris.target

print(f"数据集形状: {X.shape}")
print(f"特征名称: {iris.feature_names}")
print(f"类别名称: {iris.target_names}")

## 2. 基础 K-Means 聚类

### 关键参数说明

- `n_clusters`: 聚类数量 K
- `init`: 初始化方法，'k-means++' 是智能初始化，能加速收敛
- `n_init`: 使用不同初始质心运行算法的次数，选择最佳结果
- `max_iter`: 单次运行的最大迭代次数
- `random_state`: 随机种子，确保结果可复现

In [ ]:
# 创建 K-Means 模型
k = 3
kmeans = KMeans(
    n_clusters=k,
    init='k-means++',
    n_init=10,
    max_iter=300,
    random_state=42
)

# 拟合数据并预测簇标签
y_pred = kmeans.fit_predict(X)

print(f"预测的簇标签: {y_pred}")
print(f"\n验证 labels_ 属性: {y_pred is kmeans.labels_}")

In [ ]:
# 查看聚类中心
print("聚类中心 (质心):")
print(f"形状: {kmeans.cluster_centers_.shape}")
print(f"\n各簇质心坐标:")
for i, center in enumerate(kmeans.cluster_centers_):
    print(f"  簇 {i}: {center}")

## 3. 预测新样本

训练好的 K-Means 模型可以对新样本进行预测，将其分配到最近的簇。

In [ ]:
# 创建一些新样本
X_new = np.array([
    [5.0, 3.5, 1.5, 0.2],  # 类似 setosa
    [6.0, 2.8, 4.5, 1.3],  # 类似 versicolor
    [7.0, 3.0, 6.0, 2.0],  # 类似 virginica
])

# 预测新样本所属的簇
y_new_pred = kmeans.predict(X_new)
print(f"新样本预测的簇标签: {y_new_pred}")

## 4. 软聚类 (到各质心的距离)

`transform()` 方法返回每个样本到各个质心的距离，可用于软聚类分析。

距离越小表示样本与该簇越相似。

In [ ]:
# 计算新样本到各质心的距离
distances = kmeans.transform(X_new)

print("新样本到各簇质心的距离:")
for i, dist in enumerate(distances):
    print(f"  样本 {i}: {dist}")
    print(f"         最近的簇: {np.argmin(dist)} (距离: {dist.min():.4f})")

## 5. 模型评估

### 惯性 (Inertia)

惯性是所有样本到其所属簇质心的距离平方和，反映了簇的紧凑程度。

$$\text{Inertia} = \sum_{i=1}^{n} \|x_i - \mu_{c_i}\|^2$$

- 惯性越小，簇内越紧凑
- `score()` 返回惯性的负值 (因为 sklearn 惯例是分数越高越好)

In [ ]:
# 查看模型惯性
print(f"惯性 (Inertia): {kmeans.inertia_:.4f}")
print(f"分数 (Score): {kmeans.score(X):.4f}")
print(f"迭代次数: {kmeans.n_iter_}")

## 6. 肘部法则 (Elbow Method)

肘部法则用于确定最佳聚类数 K:

1. 对不同的 K 值运行 K-Means
2. 绘制 K 与惯性的关系曲线
3. 选择曲线出现"肘部"(拐点)的 K 值

原理: 随着 K 增加，惯性会持续下降。但当 K 达到真实簇数后，惯性下降速度会明显减缓。

In [ ]:
# 计算不同 K 值的惯性
k_range = range(1, 11)
inertias = []

for k in k_range:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X)
    inertias.append(km.inertia_)
    print(f"K={k}: Inertia={km.inertia_:.2f}")

# 绘制肘部曲线
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=3, color='r', linestyle='--', label='K=3 (肘部)')
plt.xlabel('聚类数量 K', fontsize=12)
plt.ylabel('惯性 (Inertia)', fontsize=12)
plt.title('肘部法则 - 确定最佳聚类数', fontsize=14)
plt.xticks(k_range)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. 聚类结果可视化

使用前两个特征进行二维可视化，展示聚类效果。

In [ ]:
# 使用 K=3 进行最终聚类
kmeans_final = KMeans(n_clusters=3, n_init=10, random_state=42)
y_pred_final = kmeans_final.fit_predict(X)

# 可视化 (使用前两个特征)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 真实标签
scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', 
                           edgecolors='k', s=50, alpha=0.7)
axes[0].set_xlabel(iris.feature_names[0])
axes[0].set_ylabel(iris.feature_names[1])
axes[0].set_title('真实标签')
plt.colorbar(scatter1, ax=axes[0])

# K-Means 预测
scatter2 = axes[1].scatter(X[:, 0], X[:, 1], c=y_pred_final, cmap='viridis',
                           edgecolors='k', s=50, alpha=0.7)
# 绘制质心
axes[1].scatter(kmeans_final.cluster_centers_[:, 0], 
                kmeans_final.cluster_centers_[:, 1],
                c='red', marker='X', s=200, edgecolors='k', linewidths=2,
                label='质心')
axes[1].set_xlabel(iris.feature_names[0])
axes[1].set_ylabel(iris.feature_names[1])
axes[1].set_title('K-Means 聚类结果')
axes[1].legend()
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

## 8. 合成数据实验

使用合成数据验证 K-Means 在理想情况下的表现。

In [ ]:
# 生成合成数据: 4 个明显分离的簇
X_blobs, y_blobs = make_blobs(
    n_samples=300,
    centers=4,
    cluster_std=0.6,
    random_state=42
)

# K-Means 聚类
kmeans_blobs = KMeans(n_clusters=4, n_init=10, random_state=42)
y_blobs_pred = kmeans_blobs.fit_predict(X_blobs)

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_blobs, cmap='viridis',
                edgecolors='k', s=50, alpha=0.7)
axes[0].set_title('合成数据 - 真实标签')
axes[0].set_xlabel('特征 1')
axes[0].set_ylabel('特征 2')

axes[1].scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_blobs_pred, cmap='viridis',
                edgecolors='k', s=50, alpha=0.7)
axes[1].scatter(kmeans_blobs.cluster_centers_[:, 0],
                kmeans_blobs.cluster_centers_[:, 1],
                c='red', marker='X', s=200, edgecolors='k', linewidths=2,
                label='质心')
axes[1].set_title('K-Means 聚类结果')
axes[1].set_xlabel('特征 1')
axes[1].set_ylabel('特征 2')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n惯性: {kmeans_blobs.inertia_:.2f}")

## 总结

### K-Means 优点
- 简单、高效，时间复杂度为 O(n*k*t)，其中 n 是样本数，k 是簇数，t 是迭代次数
- 易于理解和实现
- 可扩展到大规模数据集 (参见 Mini-Batch K-Means)

### K-Means 局限性
- 需要预先指定簇数 K
- 假设簇为凸形、大小相近的球状，无法处理非凸形簇
- 对初始质心敏感 (k-means++ 可缓解)
- 对异常值敏感
- 对特征尺度敏感，需要标准化

### 最佳实践
1. 数据预处理: 标准化特征
2. 使用 k-means++ 初始化
3. 多次运行选最佳 (n_init > 1)
4. 使用肘部法则或轮廓系数确定 K